## 加载需要的库

In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions


E:\Anaconda2\envs\py35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 数据读入

In [16]:
import os
import shutil

train_filenames = os.listdir('train')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir('train2')
os.mkdir('train2/cat')
os.mkdir('train2/dog')

rmrf_mkdir('test2')
#os.symlink('../test/', 'test2/test')  #windows下link不好用
#shutil.copy('test', 'test2\\test')  #自己动手复制

for filename in train_cat:
#    os.symlink('../../train/'+filename, 'train2/cat/'+filename)
    shutil.copy('train\\'+filename, 'train2\\cat\\'+filename)

for filename in train_dog:
#    os.symlink('../../train/'+filename, 'train2/dog/'+filename)
    shutil.copy('train\\'+filename, 'train2\\dog\\'+filename)
    

## 特征向量

In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)   #预处理函数

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)  #不含顶部全连接层
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    
    gen = ImageDataGenerator(rescale = 1./255)
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    
    # 训练图像的数量
    image_numbers = train_generator.samples
    print ("train_generator images numbers={}".format(image_numbers))

    # 输出类别信息
    print (train_generator.class_indices)

    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    
    # 测试图像的数量
    image_numbers = test_generator.samples
    print ("test_generator images numbers={}".format(image_numbers))

    # 输出类别信息
    print (test_generator.class_indices)

    train = model.predict_generator(train_generator, train_generator.samples)
    test = model.predict_generator(test_generator, test_generator.samples)
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)